In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import pandas as pd

In [ ]:
# For computation
import numpy as np
import math
import scipy.stats as stats
import matplotlib.pyplot as plt
from shapely import geometry

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_la = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")
sd_sj = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "car_scooter_brex_san_jose")
sd_ucb = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

In [ ]:
import importlib
importlib.reload(ezpv)

In [ ]:
pv_la = eipv.PhoneView(sd_la)

In [ ]:
pv_sj = eipv.PhoneView(sd_sj)

In [ ]:
pv_ucb = eipv.PhoneView(sd_ucb)

In [ ]:
def get_polygons(pvunp):
    polygons = []
    trips = pvunp.spec_details.curr_spec['evaluation_trips']

    for trip in trips:
        for leg in trip['legs']:
            if 'loc' in leg and leg['loc']['geometry']['type'] == 'Polygon':
                polygons.append(geometry.Polygon(leg['loc']['geometry']['coordinates'][0]))
            if 'end_loc' in leg and leg['end_loc']['geometry']['type'] == 'Polygon':
                polygons.append(geometry.Polygon(leg['end_loc']['geometry']['coordinates'][0]))
            if 'start_loc' in leg and leg['start_loc']['geometry']['type'] == 'Polygon':
                polygons.append(geometry.Polygon(leg['start_loc']['geometry']['coordinates'][0]))
    return polygons

In [ ]:
def clean(arr):
    """
    Latitude and Longitude defined as np array with shape (n,)
    Reshape to (n,1)
    """
    return arr.values.reshape((arr.shape[0], 1))

def is_point_in_polygon(point, polygons):
    """
    Utility function to check if given Shapely point
    is contained within a Shapely polygon
    """
    for polygon in polygons:
        if polygon.contains(point):
            return True
    return False

def get_trajectory(df, polygons, include_timestamps=True):
    """ 
    Returns np array combining latitudes, longitudes
    and timestamps (if asked for). Only returns those
    points that are not contained within Parking Polygons.
    """
    latitudes = clean(df['latitude'])
    longitudes = clean(df['longitude'])
    if include_timestamps:
        timestamps = clean(df['ts'])
    
    coordinates = np.concatenate((longitudes, latitudes), axis=1).tolist()
    
    data_points = []
    
    for (i, coordinate) in enumerate(coordinates):
        point = geometry.Point(coordinate[0], coordinate[1])
        
        if not is_point_in_polygon(point, polygons):
            if include_timestamps:
                coordinate.append(timestamps[i][0])
        data_points.append(coordinate)
    
    return data_points

def get_gt_points(pvunp, phone_os, phone_label, trip_index, section_index):
    """
    Get lat-long corrdinates in ground truth
    """
    data = pvunp.spec_details.curr_spec['evaluation_trips'][trip_index]['legs'][section_index]
    if 'route_coords' in data:
        return data['route_coords']['geometry']['coordinates']
    return []

def get_measured_points(df, polygons):
    """
    Get lat-long coordinates recorded from phone
    """
    return get_trajectory(df, polygons, include_timestamps=False)

def get_projections(line, measured_points):
    projections = []
    for p in measured_points:
        point = geometry.Point(p[0], p[1])
        projections.append(point.distance(line))
    return projections

def convert_to_xy(points):
    xy_points = []
    R = 6371000 # Radius of the earth in m
    
    for point in points:
        lon = point[0]
        lat = point[1]
        
        theta = math.pi/2 - math.radians(lat) 
        phi = math.radians(lon)
        
        x = R * math.sin(theta) * math.cos(phi)
        y = R * math.sin(theta) * math.sin(phi)
        xy_points.append([x, y])
        
    return xy_points

In [ ]:
def get_spatial_errors(pvunp):
    spatial_error_list = []
    polygons = get_polygons(pvunp)
    
    for phone_os, phone_map in pvunp.map().items():
        
        for phone_label, phone_detail_map in phone_map.items():
            
            for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
                run_errors = []

                for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                    trip_errors = []

                    for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                        section_gt_points = get_gt_points(pvunp, phone_os, phone_label, tr_idx, sr_idx)
                        if len(sr['location_df']) == 0:
                            continue

                        section_measured_points = get_measured_points(sr['location_df'], polygons)
                        line = geometry.LineString(convert_to_xy(section_gt_points))
                        projections = get_projections(line, convert_to_xy(section_measured_points))

                        trip_errors += projections
                    
                    run_errors += trip_errors
                
                
                spatial_error_entry = {"phone_os": phone_os, "phone_label": phone_label, "timeline": pvunp.spec_details.curr_spec["id"], "run": r_idx, "role": r["eval_role_base"], "errors": run_errors}
                spatial_error_list.append(spatial_error_entry)   
    return spatial_error_list

In [ ]:
spatial_errors_list = []
spatial_errors_list.extend(get_spatial_errors(pv_la))
spatial_errors_list.extend(get_spatial_errors(pv_sj))
spatial_errors_list.extend(get_spatial_errors(pv_ucb))

spatial_errors_df = pd.DataFrame(spatial_errors_list)

In [ ]:
r2q_map = {"power_control": 0, "HAMFDC": 1, "MAHFDC": 1, "HAHFDC": 2, "accuracy_control": 3}

In [ ]:
spatial_errors_df["quality"] = spatial_errors_df.role.apply(lambda r: r2q_map[r])
spatial_errors_df["label"] = spatial_errors_df.role.apply(lambda r: r.replace('_control', ''))
timeline_list = ["train_bus_ebike_mtv_ucb", "car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]

# Plot of Individual Errors (Over each individual run for Android)

In [ ]:
fig = plt.figure(9, figsize=(20, 15))
os = "android"

for i, tl in enumerate(timeline_list):
    for r in range(3):
        data = []
        labels = []
        ax = fig.add_subplot(3,3, (i*len(timeline_list) + r) + 1)
        for q in range(4):
            data.extend(spatial_errors_df.query('timeline == @tl & run == @r & phone_os == @os & quality == @q')['errors'])
            labels.extend(spatial_errors_df.query('timeline == @tl & run == @r & phone_os == @os & quality == @q')['role'])
        ax.set_title(tl + " - Run: " + str(r+1))
        ax.set_xticklabels(labels)
        ax.set_ylabel('Spatial Errors in Meters (android)')
        bp = ax.boxplot(data)

# Plot of Individual Errors (Over each individual run for iOS)

In [ ]:
fig = plt.figure(9, figsize=(20, 15))
os = "ios"

for i, tl in enumerate(timeline_list):
    for r in range(3):
        data = []
        labels = []
        ax = fig.add_subplot(3,3, (i*len(timeline_list) + r) + 1)
        for q in range(4):
            data.extend(spatial_errors_df.query('timeline == @tl & run == @r & phone_os == @os & quality == @q')['errors'])
            labels.extend(spatial_errors_df.query('timeline == @tl & run == @r & phone_os == @os & quality == @q')['role'])
        ax.set_title(tl + " - Run: " + str(r+1))
        ax.set_xticklabels(labels)
        ax.set_ylabel('Spatial Errors in Meters (iOS)')
        bp = ax.boxplot(data)

In [ ]:
fig = plt.figure(3, figsize=(11, 2.75))
os = "android"

for i, tl in enumerate(timeline_list):
    ax = fig.add_subplot(1,3,i+1)
    
    for r in range(3):
        data = []
        labels = []
        
        for q in range(4):
            data.extend(spatial_errors_df.query('timeline == @tl & run == @r & phone_os == @os & quality == @q')['errors'])
            labels.extend(spatial_errors_df.query('timeline == @tl & run == @r & phone_os == @os & quality == @q')['label'])
    ax.set_title(tl)
    ax.set_xticklabels(labels)
    ax.set_ylabel('Spatial Errors in Meters (android)')
    bp = ax.boxplot(data)
    plt.tight_layout()

In [ ]:
fig = plt.figure(3, figsize=(11, 2.75))
os = "ios"

for i, tl in enumerate(timeline_list):
    ax = fig.add_subplot(1,3,i+1)
    
    for r in range(3):
        data = []
        labels = []
        
        for q in range(4):
            data.extend(spatial_errors_df.query('timeline == @tl & run == @r & phone_os == @os & quality == @q')['errors'])
            labels.extend(spatial_errors_df.query('timeline == @tl & run == @r & phone_os == @os & quality == @q')['label'])
    ax.set_title(tl)
    ax.set_xticklabels(labels)
    ax.set_ylabel('Spatial Errors in Meters (android)')
    bp = ax.boxplot(data)
    plt.tight_layout()